In [1]:
# Required libraries
import pandas as pd
import numpy as np

# nltk libraries required
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Models Building
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

# Datasets Loadings

In [2]:
df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
# shape
df.shape

(50000, 2)

### Full reviews of one 

In [4]:
df['review'].iloc[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

<h4>Above reviews observations:</h4> 
    
    
1. There are <html> tags, we should remove them.
2. There are special characters, we should remove them.
3. There are stopwords, we should remove them.
4. We should convert them all into lowercase.

### Removing html tags

In [5]:
import re

def remove_tags(txt):
    remove_tag = re.compile('<.*?>')
    return re.sub(remove_tag,'',txt)

In [6]:
df['review'] = df['review'].apply(remove_tags)

#### Converting Lowercase

In [7]:
def convert_lower(txt):
    return txt.lower()

In [8]:
df['review'] = df['review'].apply(convert_lower)

#### Removing Special Characters 

In [9]:
def replace_non_characters_with_space(txt):
    replace_list = [sen if sen.isalnum() else ' ' for sen in txt]
    replace_str = ''.join(replace_list)
    return replace_str

In [10]:
df['review'] = df['review'].apply(replace_non_characters_with_space)

#### Removing Stop words

In [11]:
# Download stop_words by following code:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bohar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
stop_words = stopwords.words('english')

In [13]:
# Remove below list words from stop_words becaue they in our model so we need this world
l=['not','no','but']
for word in stop_words:
    if word in l:
        stop_words.remove(word)


In [14]:
def remove_stop_words(txt):
    with_out_stop_list = [a for a in txt.split() if a not in stop_words]
    return with_out_stop_list

In [15]:
df['review'] = df['review'].apply(remove_stop_words)

### Streaming

In [17]:
# from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [18]:
def streaming(txt):
    steam_word_list = [ps.stem(i) for i in txt ]
    string_with_streaming = ' '.join(steam_word_list)
    return string_with_streaming

In [19]:
df['review'] = df['review'].apply(streaming)

In [20]:
df['review']

0        one review mention watch 1 oz episod hook righ...
1        wonder littl product film techniqu unassum old...
2        thought wonder way spend time hot summer weeke...
3        basic famili littl boy jake think zombi closet...
4        petter mattei love time money visual stun film...
                               ...                        
49995    thought movi right good job creativ origin fir...
49996    bad plot bad dialogu bad act idiot direct anno...
49997    cathol taught parochi elementari school nun ta...
49998    go disagre previou comment side maltin one sec...
49999    no one expect star trek movi high art but fan ...
Name: review, Length: 50000, dtype: object

### CountVectorizer

In [21]:
# from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=18000) #10000,15000

In [22]:
X = cv.fit_transform(df['review']).toarray()

### LabelEncoder 

In [23]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

In [24]:
y = lb.fit_transform(df['sentiment'])

### Spliting of Datasets

In [25]:
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

### Model building

In [26]:
bb = BernoulliNB(alpha=0.1,binarize=0.0)
bb.fit(X_test, y_test)

BernoulliNB(alpha=0.1)

In [27]:
print('Accuracy:',bb.score(X_test, y_test))

Accuracy: 0.9206


In [ ]:
import pickle
pickle.dump(bb,open('nlp_model.pkl','wb'))
pickle.dump(cv,open('transform.pkl','wb'))


In [28]:
# Testing a reviws how does it work 
review =cv.transform(["Disappointing!!! Is one word i can say about the movie. Nothing makes you feel wow about the movie. It feels like every scene you have seen it somewhere in some movie or series, i think Bollywood should understand that it is not 90s crowd which watched their films blindly, today everyone has access to international and regional content at their finger tips. Please don't waste you hard earned money on this movie. Nothing against SRK but the movie is not good sorry. Deepika looks ok, it feels she is sick in some scenes not sure but it appears she is not fresh 😁. Jhon abraham is good. Overall the movie is a big disappointment is what one can say. In the times of movies like kantara and RRR this is too out dated and average."]).toarray()

In [29]:
bb.predict(review)

array([0])